# Practice 1-CNNs (Exercise 2)

##### Ovidio Manteiga Moar
##### Carlos Villar Martínez

In this first practise weare going to develop a deel learning model using the keras library, we have been given a dataset with 100000 pictures of faces and their corresponding labels. The main goal is creating a model that is capable of determining the gender of person in each photo and the age. In this part we are going to focus on the age prediction part, something important is that we can't train a new model, we must use the model we trained in the previous part.

In [1]:
# Dataset path
from os import listdir
import os.path

PATH = "C:/Users/corre/OneDrive/Escritorio/Segundo cuatri/DL/Practica 1/datasets/faces/faces_dataset_train/"
PATH_TEST = "C:/Users/corre/OneDrive/Escritorio/Segundo cuatri/DL/Practica 1/datasets/faces/faces_dataset_test/"

assert os.path.exists(PATH), 'Verify the above commands'
assert os.path.exists(PATH_TEST), 'Verify the above commands'

In [2]:
from tensorflow import keras 
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib import pyplot
import json
import os
import numpy as np
from keras.models import load_model
from keras.models import Sequential 
from tensorflow.keras.layers import Dense

### Metadata access

In [3]:
with open(PATH + 'metadata.json') as fIn:
    metadata = json.loads(fIn.read())
with open(PATH_TEST + 'metadata.json') as fIn:
    metadata_test = json.loads(fIn.read())

metadata_example_key = next(iter(metadata.keys()))
metadata_example_value = metadata[metadata_example_key]

print(metadata_example_key, end=' ----> ')
print(metadata_example_value)

i07/nm0322407_rm852269056_1955-3-29_2005.jpg ----> {'age': 50, 'gender': 'M'}


### Image reading

Again we import the needed data and create the trainning set and the test set (which again i going to be used s a validation set).

In [4]:
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'

In [5]:
# Size at which images will be processed
img_height = 32
img_width = 32

def decode_img(img):
    '''Transforms a string that represents the path of an image into a matrix
    (img_height, img_width, 3) of uint8 representing the pixel values
    '''
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size.
    return tf.cast(tf.image.resize(img, [img_height, img_width]),tf.uint8)

def process_path(file_path):
    '''Transforms strings of the form 'path|age|gender' to tuples (image, label)
    '''
    parts = tf.strings.split(file_path, '|')
    age = parts[1]
    age = tf.strings.to_number(age, out_type=tf.dtypes.int32)
    gender = parts[2]
    # Load the raw data from the file as a string
    img = tf.io.read_file(parts[0])
    img = decode_img(img)
    return img, age


def format_metadata(m, path):
    '''Transforms the metadata dictionary m into a list of strings of the form 'path to the image|age|gender'.
    '''
    return list(map(lambda x: '{0}{1}|{2}|{3}'.format(path,x,m[x]['age'],m[x]['gender']),m.keys()))


# We build a dataset of text strings from the metadata
train_metadata = format_metadata(metadata, PATH)
train_ds = tf.data.Dataset.from_tensor_slices(train_metadata)
print(train_ds.take(1))

# Mapping to an image dataset with tags
train_ds = train_ds.shuffle(len(train_ds)).map(process_path)

# Same process for test
test_ds = tf.data.Dataset.from_tensor_slices(format_metadata(metadata_test, PATH_TEST))
test_ds = test_ds.map(process_path)

# Checking
#for image, label in train_ds.take(5):
#    print("Image shape: ", image.numpy().shape)
#    print("Label: ", label.numpy())
#    pyplot.imshow(image.numpy())
#    pyplot.show()



<TakeDataset element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>


In [6]:
ds_train_transform = train_ds
ds_test_transform = test_ds
print(train_metadata[0])

C:/Users/corre/OneDrive/Escritorio/Segundo cuatri/DL/Practica 1/datasets/faces/faces_dataset_train/i07/nm0322407_rm852269056_1955-3-29_2005.jpg|50|M


In [7]:
# OPTIONAL: rescaling input to [-1, 1]
ds_train_transform = ds_train_transform.map(lambda image, label: (tf.cast(image, tf.float32)/127.5 - 1., label))
ds_test_transform = ds_test_transform.map(lambda image, label: (tf.cast(image, tf.float32)/127.5 - 1., label))

In [8]:
# OPTIONAL: rescaled output to [0, 1]
labels = [int(x.split('|')[1]) for x in train_metadata]
max_age = tf.convert_to_tensor(np.max(labels), tf.float32)
ds_train_transform = ds_train_transform.map(lambda image, label: (image, tf.cast(label, tf.float32)/max_age))
ds_test_transform = ds_test_transform.map(lambda image, label: (image, tf.cast(label, tf.float32)/max_age))

In [9]:
# OPTIONAL: sample age weights
step = 10
labels = [int(x.split('|')[1]) for x in train_metadata]
freqs, bins = np.histogram(labels, bins=list(range(0,100,step)) + [200])
i_freqs = 1. / freqs
sample_weights = tf.convert_to_tensor(len(i_freqs) * i_freqs / i_freqs.sum())
print(sample_weights)
ds_train_transform = ds_train_transform.map(lambda image, label: (image, label, sample_weights[tf.minimum(tf.cast(label, tf.int32) // step, len(i_freqs) - 1)]))
ds_test_transform = ds_test_transform.map(lambda image, label: (image, label, sample_weights[tf.minimum(tf.cast(label, tf.int32) // step, len(i_freqs) - 1)]))

tf.Tensor(
[1.76150383 0.08628557 0.01546083 0.01225852 0.01755069 0.03502511
 0.06879879 0.16056922 0.52553153 7.31701591], shape=(10,), dtype=float64)


### Batch size
Again we need to choose a batch size that adaps to our trainning set.

In [10]:
batch_size = 256
ds_train_batch = ds_train_transform.batch(batch_size)
ds_test_batch = ds_test_transform.batch(batch_size)

### Loading the old model

Here we load the trained model of the previous practise. To do it we have to import from keras 'load_model'. After loading the old model we can see it's summary which should be the same as before and, to make sure that it won't train we can define 'trainable = False' in some cell after loading it. Now, for creating the new model we delete the last layer of the old model and we add a new one and three dense layer more, as it is a regression problem two of the dense layers are using 'activation = 'relu' and the last layer is using 'activation = 'linear'.

In [11]:
#Load the pretrained model 
old_model = load_model("genre_model.h5")

In [12]:
old_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                             

In [13]:
#make it non trainable
old_model.trainable = False

In [14]:
#create the new model

new_model = Sequential()
for layer in old_model.layers[:-1]:
    new_model.add(layer)

In [15]:
# It has a single output neuron and a sigmoid activation

new_model.add(Dense(256, activation='relu'))
new_model.add(Dense(128, activation='relu'))
new_model.add(Dense(1, activation='linear'))

In [16]:
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 32, 32, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 14, 14, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                        

### Compile
Again we have to compile the model, we decided to use here the adam optimizer as it is one of the most used in regresion problems and, for the metrics, we are using mean absolute error because we are asked to use it in the statement of the practice.

In [17]:
new_model.compile(optimizer="adam",
                  loss="mse",
                  metrics=["mean_absolute_error"],
                  weighted_metrics=[])

### Training
As it's mentioned before all the part of the model that belongs to the old model can't be trained. Again we are using the the test set as validation set.

In [18]:
history = new_model.fit(ds_train_batch,
                        epochs=15,
                        validation_data=ds_test_batch)

Epoch 1/15
496/496 [==============================] - 29s 56ms/step - loss: 0.0421 - mean_absolute_error: 0.1163 - val_loss: 0.1406 - val_mean_absolute_error: 0.2355
Epoch 2/15
496/496 [==============================] - 29s 58ms/step - loss: 0.0333 - mean_absolute_error: 0.1081 - val_loss: 0.1343 - val_mean_absolute_error: 0.2301
Epoch 3/15
496/496 [==============================] - 31s 62ms/step - loss: 0.0327 - mean_absolute_error: 0.1069 - val_loss: 0.1358 - val_mean_absolute_error: 0.2338
Epoch 4/15
496/496 [==============================] - 33s 67ms/step - loss: 0.0325 - mean_absolute_error: 0.1067 - val_loss: 0.1380 - val_mean_absolute_error: 0.2357
Epoch 5/15
496/496 [==============================] - 29s 59ms/step - loss: 0.0321 - mean_absolute_error: 0.1060 - val_loss: 0.1425 - val_mean_absolute_error: 0.2375
Epoch 6/15
496/496 [==============================] - 30s 61ms/step - loss: 0.0321 - mean_absolute_error: 0.1059 - val_loss: 0.1377 - val_mean_absolute_error: 0.2341
Epoc

### Mean absolute error

Let's see now the value of the mean absolute error in years.

In [19]:
training_error = history.history["mean_absolute_error"]
validation_error = history.history["val_mean_absolute_error"]

training_error = training_error[-1]*100
validation_error = validation_error[-1]*100

print("Mean absolute error of trainning in years:", f"{training_error:.2f}", "years")
print("Mean absolute error of validation in years:", f"{validation_error:.2f}", "years")

Mean absolute error of trainning in years: 10.52 years
Mean absolute error of validation in years: 23.73 years


### Results and conclusions
The results obtained for this part are not really good, when we correct the error that there was in the original code the validation mean absolute error went from 0.10 to 0.23 which is quite disapointing. We would like to try more models to improve this result but we had no time.